In [55]:
import pandas as pd
import seaborn as sns
import matplotlib as plt
import numpy as np

In [56]:
prestamo = pd.read_csv('train.csv')

In [57]:
prestamo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1442 entries, 0 to 1441
Data columns (total 62 columns):
 #   Column                                                                                                                                                   Non-Null Count  Dtype  
---  ------                                                                                                                                                   --------------  -----  
 0   Stock code                                                                                                                                               1442 non-null   object 
 1   Z-SCORE                                                                                                                                                  1442 non-null   float64
 2   Share pledge ratio of controlling shareholders                                                                                                           1442 no

In [58]:
prestamo.describe

<bound method NDFrame.describe of      Stock code    Z-SCORE  Share pledge ratio of controlling shareholders  \
0        X00001   2.420039                                        0.152395   
1        X00002   3.844693                                        0.608351   
2        X00003   4.975553                                        0.499388   
3        X00004   4.088897                                        0.000000   
4        X00005  14.464241                                        0.332566   
...         ...        ...                                             ...   
1437     X01438   2.060600                                        1.000000   
1438     X01439   1.593100                                        1.000000   
1439     X01440   4.175800                                        0.920500   
1440     X01441  15.781100                                        0.999100   
1441     X01442   2.759900                                        1.000000   

      Pledge ratio of limited

In [59]:
df = prestamo.drop(columns=['Stock code'])

In [60]:
pd.set_option('display.max_columns', None)
df.head()

,Z-SCORE,Share pledge ratio of controlling shareholders,Pledge ratio of limited sale shares,Pledge ratio of unlimited shares,Stock Volatility,P/E ratio,P/B ratio,Stock price rise and fall in the last year,Annual turnover rate,ST,ROA,ROE,changes in net assets,changes in operating income,Gross profit margin on sales,Net business cycle (days),Turnover rate of accounts receivable (Times),Inventory turnover rate (Times),Total asset turnover rate (Times),Current ratio,Cash ratio,Asset liability ratio,Asset liability ratio (excluding advance receipts),Asset liability ratio (total liabilities - contract liabilities - advance receipts)/(total assets - goodwill - contract liabilities - advance receipts),Current liabilities/total liabilities,Monetary capital/short-term debt,EBITDA/interest bearing debt,EBIT interest cover,EBITDA interest cover,Net cash flow from operations has been negative for three consecutive years,Average cash income ratio in recent three years,Cash income ratio,Ratio of accounts receivable to operating income,Ratio of prepayments to operating income,Ratio of other receivables to total assets,Ratio of construction in progress to total assets,Minority shareholders' equity/owners' equity,The proportion of goodwill in total assets exceeds,Downgrade or negative,audit opinion,"Company nature (state owned assets 0, others 1)",High deposit and loan of 90p,Company size (LN),Whether there are four major audits,Proportion of independent directors,Equity checks and balances (2-5 large/1 large),"Two positions in one (1 for the same, 0 for the different)",Equity concentration (the first largest shareholder),Total institutional shareholding ratio,Audit fee,Tobin Q,Debt financing costs,Enterprise age,Goodwill impairment ratio,Asset quality index,SG&A Expense,Number of key audit matters,Number of research institutions concerned,Number of research reports (+1 LN),Financial cycle m2/gdp,IsDefault
0,2.420039,0.152395,0.000000,0.126366,49.318786,31.40618754,3.658663,-0.366077,1.420614,0.0,0.046191,0.129841,1.049694,0.006857,0.193685,-33.0698,22.8269,6.0945,0.7614,0.011240,0.394846,0.648161,0.632803,0.632901,0.797411,3.304139,0.501851,0.0000,0.000000,0.0,1.142042,1.158108,0.048137,0.016229,0.013032,0.044810,0.000225,0.000150,0.0,0.0,1,0,7.524870,0,0.428571,0.558561,0,0.5836,0.908206,254.716982,1.761004,-0.020707,11.258333,0.000000,1.212208,1.158201,1,3.688879,5.505332,0.028941,0
1,3.844693,0.608351,0.014874,0.159443,48.390237,30.42292058,5.460896,-0.133857,2.676361,0.0,0.121959,0.190550,0.937501,0.400794,0.727484,140.8045,3.5295,2.3282,0.4556,0.011256,0.438026,0.564089,0.559887,0.654380,0.594615,0.649839,0.326339,7.4740,8.646792,0.0,1.052153,1.050286,0.309851,0.014669,0.013802,0.034118,0.034369,0.143022,0.0,0.0,1,0,5.018160,1,0.428571,1.953691,1,0.1370,0.655950,260.000000,3.017307,0.026496,9.166667,-0.000210,1.058507,1.021779,1,3.496508,5.332719,-0.059096,0
2,4.975553,0.499388,0.000000,0.153660,46.639579,106.732585,3.636306,-0.304963,5.589544,0.0,0.015570,0.034949,0.939371,-0.166925,0.614608,103.6719,7.4687,2.9242,0.5234,0.017362,1.015669,0.383526,0.309446,0.315073,0.666706,4.003400,0.667316,2.5996,4.586201,0.0,1.061750,1.072391,0.115249,0.036500,0.005863,0.000598,0.020408,0.015943,0.0,0.0,1,0,5.138120,1,0.400000,0.818361,0,0.2533,0.567395,445.000000,2.695844,0.018959,10.175000,-0.000052,1.138692,1.260414,2,3.784190,5.318120,-0.067378,0
3,4.088897,0.000000,0.000000,0.001285,31.025124,12.81982971,3.392250,0.906496,2.538287,0.0,0.164398,0.295278,1.400732,0.355495,0.326878,116.6440,3.6095,3.9412,0.9210,0.016252,0.519451,0.497172,0.489956,0.490227,0.936333,1.255867,1.222764,258.2331,297.906926,0.0,1.063486,1.040956,0.288015,0.008373,0.030776,0.012202,0.024288,0.000546,0.0,0.0,1,0,6.808391,0,0.400000,0.512723,0,0.3000,0.604141,300.000000,2.083934,-0.001030,16.494444,-0.000043,0.929914,0.856676,2,3.663562,5.252273,-0.064161,0
4,14.464241,0.332566,0.232977,0.014730,41.572718,94.0221809,65.671415,0.986821,3.692384,0.0,0.229710,0.598016,1.245824,

In [61]:
df['P/E ratio'] = pd.to_numeric(df['P/E ratio'], errors='coerce')

In [62]:
nan_count = df['P/E ratio'].isna().sum()

nan_count

np.int64(7)

In [63]:
df = df[df['P/E ratio'].notna()]

In [74]:
X  = df[['Share pledge ratio of controlling shareholders','Pledge ratio of unlimited shares','audit opinion ', 'Downgrade or negative', 'Ratio of other receivables to total assets', 'ROA', 'Asset liability ratio (total liabilities - contract liabilities - advance receipts)/(total assets - goodwill - contract liabilities - advance receipts)', 'Pledge ratio of limited sale shares', 'ROE', 'Enterprise age']]
y = df['IsDefault']

In [ ]:
df.corr()

In [75]:
matriz_corr = df.corr()

target_corr = matriz_corr['IsDefault'].drop('IsDefault')

top_corr_columns = target_corr.abs().sort_values(ascending=False).head(10).index

threshold = 0.8
selected_columns = []
for col in top_corr_columns:
    is_highly_correlated = any(matriz_corr[col][selected_col] > threshold for selected_col in selected_columns)
    if not is_highly_correlated:
        selected_columns.append(col)

print(selected_columns)

['Share pledge ratio of controlling shareholders', 'Pledge ratio of unlimited shares', 'audit opinion ', 'Downgrade or negative', 'Ratio of other receivables to total assets', 'ROA', 'Asset liability ratio (total liabilities - contract liabilities - advance receipts)/(total assets - goodwill - contract liabilities - advance receipts)', 'Pledge ratio of limited sale shares', 'ROE', 'Enterprise age']


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = LogisticRegression(max_iter=100000, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.8863109048723898